In [1]:
import pandas as pd
import numpy as np
import random
import math
import time
import datetime

import pyltr
from sklearn.model_selection import train_test_split

## Load Data

In [2]:
# Function to sample queries
def sample_queries(df, frac):
    srch_ids = df["srch_id"].unique()[0:int(len(df)*frac)]
    return df.loc[df.srch_id.isin(srch_ids)]

# Train, validation and test set generated from the train set
_train = pd.read_csv("data/processed/_train_undersampled.csv")
_val = pd.read_csv("data/processed/_val.csv")
_test = pd.read_csv("data/processed/_test.csv")

# The actual test set that will be used to submit our final result
test = pd.read_csv("data/processed/kaggle_test.csv")

# The properties that will be used for submitting the final result
Sprops = test["prop_id"]

# To speed up execution we'll just sample for now
_train = sample_queries(_train, 1)
_val = sample_queries(_val, 0.5)
_test = sample_queries(_test, 1)

In [3]:
# Drop training specific data
for df in [_train, _val, _test]:
    df.drop(["gross_bookings_usd", "position"], axis=1, inplace=True)

_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,prop_median_price_usd,prop_std_price_usd,prop_mean_prop_review_score,prop_median_prop_review_score,prop_std_prop_review_score,srch_price_rank,srch_star_rank,srch_review_score_rank,srch_location_score1_rank,srch_location_score2_rank
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,68914,2,3.0,...,72.000,231.308865,3.0,3.0,0.0,3.0,4.0,5.0,10.0,14.0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,88218,4,3.5,...,114.885,413.886419,3.5,3.5,0.0,7.5,24.0,11.0,22.5,27.0
2,4,2012-12-31 08:59:22,5,219,NaN,NaN,219,134162,5,4.5,...,548.000,91.382003,4.5,4.5,0.0,31.0,31.0,27.0,18.0,22.0
3,4,2012-12-31 08:59:22,5,219,NaN,NaN,219,139893,2,3.0,...,143.230,18.074398,3.0,3.0,0.0,3.0,7.5,4.0,7.0,10.0
4,6,2013-06-05 12:27:51,14,100,NaN,NaN,100,52376,2,0.0,...,88.080,3.789397,0.0,0.0,0.0,1.0,3.5,1.0,3.0,3.0


## Feature Engineering

In [4]:
# Normalize price_usd according to prop and srch
for df in [_train, _val, _test, test]:
    df["price_norm_srch"] = (df.price_usd - df.srch_mean_price_usd) / df.srch_std_price_usd
    df["price_norm_prop"] = (df.price_usd - df.prop_mean_price_usd) / df.prop_std_price_usd
    
    # There are properties where std = 0 so it would give infinite as result, which is bad
    df.price_norm_prop.loc[~np.isfinite(df.price_norm_prop)] = 0

C:\Users\Richard\AppData\Local\conda\conda\envs\py35\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
# Generate price difference feature
for df in [_train, _val, _test, test]:
    df["srch_price_usd_diff"] = df["price_usd"] - df["srch_mean_price_usd"]
    #df["prop_starrating_diff"] = df["prop_starrating"] - df["prop_starrating_mean"] # Seems to be a bad feature

In [6]:
# Generate month feature
for df in [_train, _val, _test, test]:
    month = df.date_time.map(lambda d: d.split("-")[1])
    dummies = pd.get_dummies(month, prefix="month")
    df[dummies.columns] = dummies

In [7]:
# Get date_time object from date_time string, more efficient than strptime
def get_date_time(s):
    year = int(s[0:4])
    month = int(s[5:7])
    day = int(s[8:10])
    return datetime.datetime(year, month, day)

# Generate target month feature
def row_to_target_month(row):
    visit_date = get_date_time(row.date_time)
    target_date = visit_date + datetime.timedelta(days=int(row.srch_booking_window + row.srch_length_of_stay/2))
    return target_date.strftime('%m')

for df in [_train, _val, _test, test]:
    if (len(df) > 0): # This is just cause sometimes I use an empty test set to speed things up
        target_month = df.apply(row_to_target_month, axis=1)
        dummies = pd.get_dummies(target_month, prefix="target_month")
        df[dummies.columns] = dummies

In [8]:
# Composite visitor features
for df in [_train, _val, _test, test]:
    df["srch_visitor_count"] = df.srch_adults_count + df.srch_children_count
    df["srch_visitor_per_room_count"] = df.srch_visitor_count / df.srch_room_count
    df["srch_price_per_adult_count"] = df.price_usd / df.srch_adults_count

In [9]:
# Composite history features
for df in [_train, _val, _test, test]:
    df.visitor_hist_starrating.fillna(df.visitor_hist_starrating.median())
    df.visitor_hist_adr_usd.fillna(df.visitor_hist_adr_usd.median())
    
    df["visitor_hist_price_diff"] = abs(df.visitor_hist_adr_usd - df.price_usd)
    df["visitor_hist_star_diff"] = abs(df.visitor_hist_starrating - df.prop_starrating)

In [10]:
# Generate target label
for df in [_train, _val, _test]:
    df["relevance"] = df.click_bool + df.booking_bool
    df.relevance = df.relevance.map(lambda r: 5 if r > 1 else r)
    df.drop(["click_bool", "booking_bool"], axis=1, inplace=True)

In [11]:
# Fill missing values for prop_location_score2
for df in [_train, _val, _test, test]:
    rounded_scores = df.prop_location_score1.map(lambda s: int(s))
    for score1 in rounded_scores.unique():
        median_score2 = df.loc[rounded_scores == score1].prop_location_score2.quantile(0.25)
        df.loc[(rounded_scores == score1) & (df.prop_location_score2.isnull())].prop_location_score2 = median_score2

# Fill remaining missing values (need better methods for each feature)
for df in [_train, _val, _test, test]:
    df.fillna(0, inplace=True)

C:\Users\Richard\AppData\Local\conda\conda\envs\py35\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


## Training and Testing

In [12]:
def split_X_y_qids(df):
    X = df.drop(["srch_id", "relevance"], axis=1)
    y = df.relevance
    qids = df.srch_id
    
    # Remove features we don't want to include
    X = X.drop(["date_time", 
                #"visitor_hist_starrating", 
                #"visitor_hist_adr_usd", 
                "prop_id", 
                "random_bool",
                "price_usd"
               ], axis=1)
    
    return (X, y, qids)
    
TX, Ty, Tqids = split_X_y_qids(_train)
VX, Vy, Vqids = split_X_y_qids(_val)
EX, Ey, Eqids = split_X_y_qids(_test)

# Train model
metric = pyltr.metrics.NDCG(k=38)

monitor = pyltr.models.monitors.ValidationMonitor(
    VX, Vy, Vqids, metric=metric, stop_after=100)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=2500,
    #max_features=1,
    query_subsample=0.5,
    max_leaf_nodes=15,
    min_samples_leaf=64,
    verbose=1,
    max_depth=6
)

start = time.time()

model.fit(TX, Ty, Tqids, monitor=monitor)

print("Time elapsed:", (time.time() - start)/60)

# Test model locally
Epred = model.predict(EX)
metric.calc_mean(Eqids, Ey.as_matrix(), Epred)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.8246       0.2022     1259.53m      C:      0.3630 B:      0.3630 S:  0
    2       0.8482       0.0228     1139.19m      C:      0.4061 B:      0.4061 S:  0
    3       0.8587       0.0104     1094.18m      C:      0.4255 B:      0.4255 S:  0
    4       0.8666       0.0075     1073.74m      C:      0.4450 B:      0.4450 S:  0
    5       0.8707       0.0050     1059.90m      C:      0.4538 B:      0.4538 S:  0
    6       0.8741       0.0028     1051.18m      C:      0.4639 B:      0.4639 S:  0
    7       0.8756       0.0015     1044.33m      C:      0.4681 B:      0.4681 S:  0
    8       0.8770       0.0013     1039.42m      C:      0.4711 B:      0.4711 S:  0
    9       0.8780       0.0015     1034.39m      C:      0.4739 B:      0.4739 S:  0
   10       0.8799       0.0013     1031.18m      C:      0.4772 B:      0.4772 S:  0
   15       0.8841       0.0007     1020.91m      C: 

0.51172614941053496

In [13]:
# Feature importances
fi = pd.DataFrame()
fi["Features"] = TX.columns
fi["Importance"] = model.feature_importances_
fi.sort("Importance", ascending=False)

C:\Users\Richard\AppData\Local\conda\conda\envs\py35\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Features,Importance
9,prop_location_score2,0.054793
74,price_norm_srch,0.052372
75,price_norm_prop,0.051605
65,prop_std_price_usd,0.049539
64,prop_median_price_usd,0.044897
76,srch_price_usd_diff,0.042715
63,prop_mean_price_usd,0.041167
10,prop_log_historical_price,0.037613
8,prop_location_score1,0.031990
99,srch_price_per_adult_count,0.030653


## Submit Final Result

In [14]:
# Predict final submission order
SX = test[TX.columns]
Sqids = test["srch_id"]
Spred = model.predict(SX)

In [17]:
# Create submission data frame
result = pd.DataFrame()
result["SearchId"] = Sqids
result["PropertyId"] = Sprops
result["Sort"] = Spred
result = result.sort(["SearchId", "Sort"], ascending=[True, False])
result

C:\Users\Richard\AppData\Local\conda\conda\envs\py35\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,SearchId,PropertyId,Sort
5,2,25579,0.574593
6,2,26540,0.222322
2,2,7374,0.147213
4,2,12938,-0.205648
10,2,78858,-0.453253
1,2,6399,-0.527060
8,2,30434,-0.561210
12,2,131173,-0.628220
11,2,91899,-0.881569
7,2,27090,-0.889026


In [16]:
# Write submission to file
result = result.drop("Sort", axis=1)
result.to_csv("submission.csv", index=False)